In [51]:
import pandas as pd

df = pd.read_csv('preprocessed_processed_image.csv')
df

,선종,상태,state,vessel_type,nation_type
0,낚시어선,조업,0,8,1
1,범장망,조업,0,3,0
2,낚시어선,표류,2,8,1
3,등광조망,이동,1,2,0
4,등광조망,표류,2,2,0
...,...,...,...,...,...
1576,안강망,조업,0,4,1
1577,연승,표류,2,5,1
1578,등광조망,조업,0,2,0
1579,등광조망,조업,0,2,0


In [52]:
# import os
# import numpy as np
# from PIL import Image, ImageEnhance, ImageFilter
# import tensorflow as tf
# from tensorflow.keras.preprocessing.image import img_to_array
# from tensorflow.keras.utils import to_categorical


# # 이미지 디렉토리 경로
# image_dir = 'imgdata'  # 경로 수정 필요

# # 이미지 읽기 및 전처리
# images = []
# for filename in os.listdir(image_dir):
#     if filename.endswith(('.jpg', '.png')):  # 여러 확장자를 처리
#         img_path = os.path.join(image_dir, filename)
#         try:
#             image = Image.open(img_path).convert('RGB').resize((256, 256))  # 이미지 크기 조정 및 RGB 변환
#             # 대비 증가
#             # enhancer = ImageEnhance.Contrast(image)
#             # image = enhancer.enhance(2.0)
            
#             image_array = img_to_array(image) / 255.0  # 정규화
#             images.append(image_array)
#         except Exception as e:
#             print(f"Error loading image {filename}: {e}")

# # 이미지 데이터를 NumPy 배열로 변환
# images = np.array(images)

# train_data = images
# train_data
# target = df['state']
# target = to_categorical(target, num_classes=3)
# target

In [53]:
# !pip install imbalanced-learn

In [75]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from imblearn.over_sampling import RandomOverSampler

# 이미지 디렉토리 경로
image_dir = 'imgdata'  # 경로 수정 필요

# 'state' 열이 존재하는지 확인하고, 없으면 KeyError 발생 방지
target_column = 'state'
if target_column not in df.columns:
    raise KeyError(f"The column '{target_column}' does not exist in the CSV file.")

# 이미지 읽기 및 전처리
images = []
targets = df[target_column].values  # 타겟 값은 데이터프레임에서 가져옴
for filename in os.listdir(image_dir):
    if filename.endswith(('.jpg', '.png')):  # 여러 확장자를 처리
        img_path = os.path.join(image_dir, filename)
        try:
            image = Image.open(img_path).convert('RGB').resize((256, 256))  # 이미지 크기 조정 및 RGB 변환
            image_array = img_to_array(image) / 255.0  # 정규화
            images.append(image_array)
        except Exception as e:
            print(f"Error loading image {filename}: {e}")

# 이미지 데이터를 NumPy 배열로 변환
images = np.array(images)

# 오버샘플링 적용을 위해 이미지를 2D 형태로 변환 (샘플 수, -1)
nsamples, nx, ny, nz = images.shape
images_reshaped = images.reshape((nsamples, nx * ny * nz))

# 오버샘플링 적용
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(images_reshaped, targets)

# 다시 원래 이미지 형태로 변환
X_resampled = X_resampled.reshape((-1, nx, ny, nz))

# 타겟을 one-hot 인코딩
y_resampled = to_categorical(y_resampled, num_classes=3)

# 최종 데이터셋
train_data = X_resampled
train_target = y_resampled

print(f"Original dataset shape: {targets.shape}")
print(f"Resampled dataset shape: {y_resampled.shape}")
print(f"Resampled data shape: {X_resampled.shape}")

Original dataset shape: (1581,)
Resampled dataset shape: (2007, 3)
Resampled data shape: (2007, 256, 256, 3)


In [76]:
print(np.unique(targets))

[0 1 2]


In [77]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def ResNet50(input_shape=(256, 256, 3), num_classes=3):
    inputs = tf.keras.Input(shape=input_shape)
    
    # ResNetV2 backbone
    backbone = tf.keras.applications.ResNet50(input_shape=input_shape, include_top=False, weights='imagenet')(inputs)
    
    # Add custom head for classification
    x = layers.GlobalAveragePooling2D()(backbone)
    x = layers.Dense(128, activation='softmax')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs, name='ResNet50')
    return model

model = ResNet50()
model.summary()

Model: "ResNet50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 8, 8, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,850,371 (90.98 MB)

 Trainable params: 23,797,251 (90.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [78]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [81]:
#20 20 30 50
# 50 50 50 50
history = model.fit(train_data, train_target, epochs=50, verbose=1)

Epoch 1/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.5437 - loss: 0.8846
Epoch 2/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.5483 - loss: 0.8710
Epoch 3/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.5675 - loss: 0.8410
Epoch 4/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.5545 - loss: 0.8707
Epoch 5/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.5442 - loss: 0.8700
Epoch 6/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.5614 - loss: 0.8456
Epoch 7/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.5374 - loss: 0.8764
Epoch 8/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.5681 - loss: 0.8336
Epoch 9/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.6010 - loss: 0.8007
Epoch 10/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 86ms/step - accuracy: 0.5448 - loss: 0.8825
Epoch 11/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.4811 - loss: 1.0103
Epoch 12/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy:

In [ ]:
# 모델 HDF5 형식으로 저장
model.save('resnet50_model.h5')

In [ ]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array

test_image_dir = '테스트셋'  # 이미지 폴더 경로

# 이미지 읽기 및 전처리
test_images = []  # 리스트로 선언

# 파일 목록을 가져오고 숫자 순서대로 정렬
file_list = sorted(os.listdir(test_image_dir), key=lambda x: int(os.path.splitext(x)[0]))

for filename in file_list:
    if filename.lower().endswith(('.jpg', '.png')):  # 확장자를 소문자로 변환 후 비교
        img_path = os.path.join(test_image_dir, filename)
        try:
            image = Image.open(img_path).convert('RGB').resize((256, 256))  # 이미지 크기 조정 및 RGB 변환
            image_array = img_to_array(image) / 255.0  # 이미지를 배열로 변환하고 정규화
            test_images.append(image_array)  # 리스트에 이미지 추가
        except Exception as e:
            print(f"Error loading image {filename}: {e}")

# 이미지 데이터를 NumPy 배열로 변환
test_images = np.array(test_images)  # 리스트를 한 번에 NumPy 배열로 변환

test_data = test_images
print(test_data)


[[[[0.7764706  0.75686276 0.73333335]
   [0.3882353  0.36862746 0.3529412 ]
   [0.26666668 0.24705882 0.24705882]
   ...
   [0.26666668 0.25882354 0.26666668]
   [0.26666668 0.25882354 0.27058825]
   [0.27058825 0.2627451  0.2784314 ]]

  [[0.7764706  0.7607843  0.7372549 ]
   [0.21568628 0.20392157 0.19215687]
   [0.         0.         0.        ]
   ...
   [0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.00392157]]

  [[0.7764706  0.7647059  0.7372549 ]
   [0.23137255 0.21960784 0.2       ]
   [0.01960784 0.00392157 0.00392157]
   ...
   [0.00784314 0.         0.00784314]
   [0.01176471 0.00392157 0.01568628]
   [0.01568628 0.00784314 0.02352941]]

  ...

  [[0.76862746 0.77254903 0.7411765 ]
   [0.21568628 0.21568628 0.20392157]
   [0.         0.         0.        ]
   ...
   [0.00784314 0.         0.01568628]
   [0.00784314 0.         0.02352941]
   [0.01176471 0.         0.02745098]]

  [[0.76862746 0.77254903 0.7411765 ]
   [0.2

In [ ]:
predictions = model.predict(test_data)
# binary_predictions = (predictions > 0.5).astype(int)
class_predictions = np.argmax(predictions, axis=1)

for filename, prediction in zip(file_list, class_predictions):
    print(f"File: {filename}, Prediction: {prediction}")


#0 조업 1이동 2표류

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
File: 1.jpg, Prediction: 1
File: 2.jpg, Prediction: 1
File: 3.jpg, Prediction: 0
File: 4.jpg, Prediction: 2
File: 5.jpg, Prediction: 2
File: 6.jpg, Prediction: 2
File: 7.jpg, Prediction: 1
File: 8.jpg, Prediction: 0
File: 9.jpg, Prediction: 2
File: 10.jpg, Prediction: 0
File: 11.jpg, Prediction: 1
File: 12.jpg, Prediction: 2
File: 13.jpg, Prediction: 0
File: 14.jpg, Prediction: 1
File: 15.jpg, Prediction: 1
File: 16.jpg, Prediction: 0
File: 17.jpg, Prediction: 1
File: 18.jpg, Prediction: 2
File: 19.jpg, Prediction: 1
File: 20.jpg, Prediction: 2
File: 21.jpg, Prediction: 2
File: 22.jpg, Prediction: 1
File: 23.jpg, Prediction: 2
File: 24.jpg, Prediction: 1
File: 25.jpg, Prediction: 0
File: 26.jpg, Prediction: 0
File: 27.jpg, Prediction: 0
File: 28.jpg, Prediction: 2
File: 29.jpg, Prediction: 2
File: 30.jpg, Prediction: 2
File: 31.jpg, Prediction: 2
File: 32.jpg, Prediction: 0
File: 33.jpg, Prediction: 2
File: 34.jpg, Prediction: 0
File: 35.jpg, Predi